# Objective:

Create Dengue database metadata

# Built environment

In [21]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import DateType, StringType
from pyspark.sql.functions import year, month, col, sum, udf, substring, split, regexp_replace, when
import glob as gb
#from tqdm import tqdm
import re 
from functools import reduce
import numpy as np
import sys
import glob

In [2]:
sc = SparkContext()
sc.stop()

conf = SparkConf().setAppName("SINAM_dengue_cases")

conf = (conf.setMaster("local[*]")
       .set("spark.executor.memory", "3GB")
       .set("spark.driver.memory", "20GB"))

sc = SparkContext(conf = conf)
spark = SparkSession(sc)
spark

In [3]:
#define paths to files
spark = SparkSession.builder.appName("Dengue").getOrCreate()

# Save columns names of each dengue dataset

In [4]:
path = '/Users/julianeoliveira/Downloads/Datalake Dengue/Sinan_dengue/juliane/'
filenames = glob.glob(path + "/*.csv")
filenames

['/Users/julianeoliveira/Downloads/Datalake Dengue/Sinan_dengue/juliane/DENGBR21.csv',
 '/Users/julianeoliveira/Downloads/Datalake Dengue/Sinan_dengue/juliane/DENGBR09.csv',
 '/Users/julianeoliveira/Downloads/Datalake Dengue/Sinan_dengue/juliane/DENGBR08.csv',
 '/Users/julianeoliveira/Downloads/Datalake Dengue/Sinan_dengue/juliane/DENGBR20.csv',
 '/Users/julianeoliveira/Downloads/Datalake Dengue/Sinan_dengue/juliane/DENGBR18.csv',
 '/Users/julianeoliveira/Downloads/Datalake Dengue/Sinan_dengue/juliane/DENGBR19.csv',
 '/Users/julianeoliveira/Downloads/Datalake Dengue/Sinan_dengue/juliane/DENGBR00.csv',
 '/Users/julianeoliveira/Downloads/Datalake Dengue/Sinan_dengue/juliane/DENGBR14.csv',
 '/Users/julianeoliveira/Downloads/Datalake Dengue/Sinan_dengue/juliane/DENGBR15.csv',
 '/Users/julianeoliveira/Downloads/Datalake Dengue/Sinan_dengue/juliane/DENGBR01.csv',
 '/Users/julianeoliveira/Downloads/Datalake Dengue/Sinan_dengue/juliane/DENGBR17.csv',
 '/Users/julianeoliveira/Downloads/Datalake

In [33]:
lst_df_columns = []
for f in filenames:
    print(f)
    #df = pd.read_csv(f, low_memory = False, encoding="iso-8859-1")  
    df = spark.read.format("csv").option("header",True).load(f)
    lst_df_columns.append([f[-12:-4]] + df.columns)

/Users/julianeoliveira/Downloads/Datalake Dengue/Sinan_dengue/juliane/DENGBR21.csv
/Users/julianeoliveira/Downloads/Datalake Dengue/Sinan_dengue/juliane/DENGBR09.csv
/Users/julianeoliveira/Downloads/Datalake Dengue/Sinan_dengue/juliane/DENGBR08.csv
/Users/julianeoliveira/Downloads/Datalake Dengue/Sinan_dengue/juliane/DENGBR20.csv
/Users/julianeoliveira/Downloads/Datalake Dengue/Sinan_dengue/juliane/DENGBR18.csv
/Users/julianeoliveira/Downloads/Datalake Dengue/Sinan_dengue/juliane/DENGBR19.csv
/Users/julianeoliveira/Downloads/Datalake Dengue/Sinan_dengue/juliane/DENGBR00.csv
/Users/julianeoliveira/Downloads/Datalake Dengue/Sinan_dengue/juliane/DENGBR14.csv
/Users/julianeoliveira/Downloads/Datalake Dengue/Sinan_dengue/juliane/DENGBR15.csv
/Users/julianeoliveira/Downloads/Datalake Dengue/Sinan_dengue/juliane/DENGBR01.csv
/Users/julianeoliveira/Downloads/Datalake Dengue/Sinan_dengue/juliane/DENGBR17.csv
/Users/julianeoliveira/Downloads/Datalake Dengue/Sinan_dengue/juliane/DENGBR03.csv
/Use

In [34]:
lst_df_columns

[['DENGBR21',
  'TP_NOT',
  'ID_AGRAVO',
  'DT_NOTIFIC',
  'SEM_NOT',
  'NU_ANO',
  'SG_UF_NOT',
  'ID_MUNICIP',
  'ID_REGIONA',
  'ID_UNIDADE',
  'DT_SIN_PRI',
  'SEM_PRI',
  'DT_NASC',
  'NU_IDADE_N',
  'CS_SEXO',
  'CS_GESTANT',
  'CS_RACA',
  'CS_ESCOL_N',
  'SG_UF',
  'ID_MN_RESI',
  'ID_RG_RESI',
  'ID_PAIS',
  'DT_INVEST',
  'ID_OCUPA_N',
  'FEBRE',
  'MIALGIA',
  'CEFALEIA',
  'EXANTEMA',
  'VOMITO',
  'NAUSEA',
  'DOR_COSTAS',
  'CONJUNTVIT',
  'ARTRITE',
  'ARTRALGIA',
  'PETEQUIA_N',
  'LEUCOPENIA',
  'LACO',
  'DOR_RETRO',
  'DIABETES',
  'HEMATOLOG',
  'HEPATOPAT',
  'RENAL',
  'HIPERTENSA',
  'ACIDO_PEPT',
  'AUTO_IMUNE',
  'DT_CHIK_S1',
  'DT_CHIK_S2',
  'DT_PRNT',
  'RES_CHIKS1',
  'RES_CHIKS2',
  'RESUL_PRNT',
  'DT_SORO',
  'RESUL_SORO',
  'DT_NS1',
  'RESUL_NS1',
  'DT_VIRAL',
  'RESUL_VI_N',
  'DT_PCR',
  'RESUL_PCR_',
  'SOROTIPO',
  'HISTOPA_N',
  'IMUNOH_N',
  'HOSPITALIZ',
  'DT_INTERNA',
  'UF',
  'MUNICIPIO',
  'TPAUTOCTO',
  'COUFINF',
  'COPAISINF',
  'COMUN

In [39]:
#search variable function

def check_variables(variable): 

    for value in lst_df_columns:
        print(value[0], variable in value)

In [119]:
check_variables('CRITERIO')

DENGBR21 True
DENGBR09 True
DENGBR08 True
DENGBR20 True
DENGBR18 True
DENGBR19 True
DENGBR00 False
DENGBR14 True
DENGBR15 True
DENGBR01 False
DENGBR17 True
DENGBR03 False
DENGBR02 False
DENGBR16 True
DENGBR12 True
DENGBR06 False
DENGBR07 True
DENGBR13 True
DENGBR05 False
DENGBR11 True
DENGBR10 True
DENGBR04 False


In [111]:
lst_df_columns[6];

In [ ]:
['DENGBR00',
 'ID_DG_NOT',
 'ID_EV_NOT',
 'ANT_DT_INV',
 'OCUPACAO',
 'DENGUE',
 'ANO',
 'VACINADO',
 'DT_DOSE',
 'FEBRE',
 'DT_FEBRE',
 'DURACAO',
 'LACO',
 'CEFALEIA',
 'EXANTEMA',
 'DOR',
 'PROSTACAO',
 'MIALGIA',
 'NAUSEAS',
 'ARTRALGIA',
 'DIARREIA',
 'OUTROS',
 'SIN_OUT',
 'EPISTAXE',
 'PETEQUIAS',
 'GENGIVO',
 'METRO',
 'HEMATURA',
 'SANGRAM',
 'OUTROS_M',
 'OUTROS_M_D',
 'ASCITE',
 'PLEURAL',
 'PERICARDI',
 'ABDOMINAL',
 'HEPATO',
 'MIOCARDI',
 'HIPOTENSAO',
 'CHOQUE',
 'MANIFESTA',
 'INSUFICIEN',
 'OUTRO_S',
 'OUTRO_S_D',
 'ASCITE',
 'PLEURAL',
 'PERICARDI',
 'ABDOMINAL',
 'HEPATO',
 'MIOCARDI',
 'HIPOTENSAO',
 'CHOQUE',
 'MANIFESTA',
 'INSUFICIEN',
 'OUTRO_S',
 'OUTRO_S_D',
 'DT_CHOQUE',
 'HOSPITALIZ',
 'DT_INTERNA',
 'UF',
 'MUNICIPIO',
 'DT_COL_HEM',
 'HEMA_MAIOR',
 'DT_COL_PLQ',
 'PALQ_MAIOR',
 'DT_COL_HE2',
 'HEMA_MENOR',
 'DT_COL_PL2',
 'PLAQ_MENOR',
 'DT_SORO1',
 'DT_SORO2',
 'DT_SOROR1',
 'DT_SOROR2',
 'S1_IGM',
 'S1_IGG',
 'S1_TIT1',
 'S2_TIT1',
 'MATERIAL',
 'SORO1',
 'SORO2',
 'TECIDOS',
 'RESUL_VIRA',
 'HISTOPA',
 'IMUNOH',
 'AMOS_PCR',
 'RESUL_PCR',
 'AMOS_OUT',
 'TECNICA',
 'RESUL_OUT',
 'CON_FHD',
 'CON_INF_MU',
 'CON_INF_UF',
 'CON_INF_PA',
 'CON_DOENCA',
 'CON_EVOLUC',
 'CON_DT_OBI',
 'CON_DT_ENC',
 'IN_AIDS']

# Check columns values

In [116]:
df = spark.read.format("csv").option("header",True).load(filenames[6])

In [117]:
df.select("CON_CLASSI").show(20)

+----------+
|CON_CLASSI|
+----------+
|      b'1'|
|      b'1'|
|      b'5'|
|      b'5'|
|      b'5'|
|      b'5'|
|      b'1'|
|      b'1'|
|      b'1'|
|      b'5'|
|      b'5'|
|      b'5'|
|      b'1'|
|      b'1'|
|      b'5'|
|      b'1'|
|      b'5'|
|      b'5'|
|      b'5'|
|      b'5'|
+----------+
only showing top 20 rows



In [118]:
df.select("CON_CLASSI").distinct().collect()

[Row(CON_CLASSI="b'2'"),
 Row(CON_CLASSI="b'5'"),
 Row(CON_CLASSI="b'3'"),
 Row(CON_CLASSI="b'0'"),
 Row(CON_CLASSI="b'4'"),
 Row(CON_CLASSI="b'8'"),
 Row(CON_CLASSI="b'1'"),
 Row(CON_CLASSI="b' '")]